### Train optimizer
Notebook to train different optimizer to compare

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
import sys
import time
import torch
import torch.optim as optim
import torch.nn as nn
from tqdm.notebook import trange, tqdm

In [3]:
# Run this cell only if you run this notebook on Google Colab.
from google.colab import drive
drive.mount('/content/drive')

sys.path.append('/content/drive/My Drive/OptiForML2022')

Mounted at /content/drive


In [4]:
from optimizer.AdaHessian import AdaHessian
from optimizer.Atmo import Atmo, MASScheduler
from optimizer.Padam_SGD import PadamSGDWeighted
from dataset import ImagesDataset
from model import ResNet18
from path import TRAIN_HISTORY_DIR, TRAIN_MODEL_DIR

In [5]:
# choose dataset
# data_name = "MNIST"
data_name = "cifar"
full = True
tiny = False

# choose optimizer
#optimizer_name = "adam"
#optimizer_name = "sgd"
#optimizer_name = "atmo"
optimizer_name = "dynamicAtom"
# optimizer_name = "adaHessian"
# optimizer_name = "padam_sgd"
#optimizer_name = "padam_sgd_dyn"

# choose scheduler
#scheduler_name = None
#scheduler_name = "cosineAnnealinglr"
#scheduler_name = "steplr"
scheduler_name = "multilr"

# choose nb of epochs
epochs = 200

In [6]:
# load dataset
train_dataset = ImagesDataset(full = full, tiny=tiny, cifar=(data_name=="cifar"))
test_dataset = ImagesDataset(full = full, tiny=tiny, cifar=(data_name=="cifar"), test=True)


trainDataLoader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True)
testDataLoader = torch.utils.data.DataLoader(test_dataset, batch_size=128, shuffle=True)

* Using CIFAR


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar10/cifar-10-python.tar.gz to ./data/cifar10/
** Use 50000 train samples
* Using CIFAR
Files already downloaded and verified
** Use 10000 test samples


In [7]:
# load model
model = ResNet18(in_channel=1 if data_name=="MNIST" else 3)
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
print(device)
model.train()
model.to(device)

# load optimizer
if optimizer_name == "adam":
    optimizer = optim.Adam(model.parameters(), lr=1e-4)
if optimizer_name == "sgd":
    optimizer = optim.SGD(model.parameters(), lr=1e-3)
if optimizer_name == "atmo":
    optimizer = Atmo(model.parameters())
if optimizer_name == "dynamicAtom":
    optimizer = Atmo(model.parameters(), adam_w=1, sgd_w=0, lr=0.01, weight_decay=2.5e-2, momentum=0.9)
    dynamic = MASScheduler(optimizer, 1, 0, 100)
if optimizer_name == "adaHessian":
    optimizer = AdaHessian(model.parameters())
if optimizer_name == "padam_sgd":
    optimizer = PadamSGDWeighted(model.parameters(), adam_w=0.5, sgd_w=0.5, lr=0.01, weight_decay=2.5e-2, partial=0.125, momentum=0.9)
if optimizer_name == "padam_sgd_dyn":
    optimizer = PadamSGDWeighted(model.parameters(), adam_w=1, sgd_w=0, lr=0.01, weight_decay=2.5e-2, partial=0.125, momentum=0.9)
    dynamic = MASScheduler(optimizer, 1, 0, 100)

# load scheduler
if scheduler_name is not None:
    if scheduler_name == "cosineAnnealinglr":
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
    if scheduler_name == "steplr":
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=150)
    if scheduler_name == "multilr":
      scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[100, 150], gamma=0.1)


criterion = nn.CrossEntropyLoss()

cuda


In [8]:
train_loss = []
train_acc = []
test_loss = []
test_acc = []
time_epoch = []

with trange(1, epochs + 1, desc='Training', unit='epoch') as t:
  for epoch in t:
    losses = []
    acc = []
    start_time = time.time()
    with tqdm(trainDataLoader, desc=f'Train epoch {epoch}',
              unit='batch', leave=False) as t1:
      for x_train, y_train in t1:
        x_train = x_train.to(device)
        y_train = y_train.to(device)

        optimizer.zero_grad()
        output = model(x_train)
        loss = criterion(output, y_train)
        loss.backward(create_graph=(optimizer_name=="adaHessian"))
        optimizer.step()

        losses.append(loss)
        pred = torch.argmax(output, axis = 1)
        acc.append(sum(pred == y_train).item()/pred.shape[0])

    if optimizer_name in ["dynamicAtmo", "padam_sgd_dyn"]:
          dynamic.step()

    train_loss.append(sum(losses)/len(losses))
    train_acc.append(sum(acc)/len(acc))

    losses = []
    acc = []
    with torch.no_grad():
      with tqdm(testDataLoader, desc=f'Test epoch {epoch}',
                unit='batch', leave=False) as t1:
        for x_test, y_test in t1:
          x_test = x_test.to(device)
          y_test = y_test.to(device)

          output = model(x_test)
          loss = criterion(output, y_test)
          losses.append(loss)

          pred = torch.argmax(output, axis = 1)
          acc.append(sum(pred == y_test).item()/pred.shape[0])

      test_loss.append(sum(losses)/len(losses))
      test_acc.append(sum(acc)/len(acc))

      end_time = time.time()
      time_epoch.append(end_time-start_time)
    if scheduler_name is not None:
          scheduler.step()

#mean by epoch
time_epoch = sum(time_epoch)/len(time_epoch)

Training:   0%|          | 0/200 [00:00<?, ?epoch/s]

Train epoch 1:   0%|          | 0/391 [00:00<?, ?batch/s]

Test epoch 1:   0%|          | 0/79 [00:00<?, ?batch/s]

Train epoch 2:   0%|          | 0/391 [00:00<?, ?batch/s]

Test epoch 2:   0%|          | 0/79 [00:00<?, ?batch/s]

...

Test epoch 198:   0%|          | 0/79 [00:00<?, ?batch/s]

Train epoch 199:   0%|          | 0/391 [00:00<?, ?batch/s]

Test epoch 199:   0%|          | 0/79 [00:00<?, ?batch/s]

Train epoch 200:   0%|          | 0/391 [00:00<?, ?batch/s]

Test epoch 200:   0%|          | 0/79 [00:00<?, ?batch/s]

In [9]:
# run this cell to save history
history = dict()
history["train_loss"] = train_loss
history["train_acc"] = train_acc
history["test_loss"] = test_loss
history["test_acc"] = test_acc
history["time_epoch"] = time_epoch
history["data"] = data_name
history["model"] = "resnet18"
history["epochs"] = epochs
history["optimizer"] = optimizer_name
with open(os.path.join(TRAIN_HISTORY_DIR, f"log-{optimizer_name}_{epochs}epochs_{scheduler_name}_{data_name}.pickle"), 'wb') as f:
  pickle.dump(history, f)

In [10]:
# run this cell to save model
weights = model.state_dict()
torch.save(weights,os.path.join(TRAIN_MODEL_DIR, f"{optimizer_name}_{epochs}epochs_{scheduler_name}_{data_name}.pt"))